In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


# 1. Data Prep

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score

# LOAD DATA
PATH = '../input'
df = pd.read_csv(f'{PATH}/train.csv')
X, y = df.iloc[:500000,1], df.iloc[:500000,2]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)
print(X_train.shape, y_train.shape)

# LOAD STOPWORDS
import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
len(stopwords)

(375000,) (375000,)


179

# 2. Vectorize

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
vectorizer = TfidfVectorizer(strip_accents=None,lowercase=True, preprocessor=None, tokenizer=None, analyzer='word', 
                             stop_words=stopwords,token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 3), max_df=1.0, min_df=1,
                             max_features=None, vocabulary=None, binary=False, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

# Fit vectorizer
vectorizer.fit(X_train)

# Transform training and test sets
X_train_vect = vectorizer.transform(X_train)
X_val_vect = vectorizer.transform(X_test)

In [53]:
X_train_vect.shape

(375000, 3009360)

# 3. Run Models

- **class sklearn.svm.SVC**(C=1.0, kernel=’rbf’, degree=3, gamma=’auto_deprecated’, coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=’ovr’, random_state=None)[source]¶
- **class sklearn.ensemble.RandomForestClassifier**(n_estimators=’warn’, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None)
- **class sklearn.ensemble.GradientBoostingClassifier**(loss=’deviance’, learning_rate=0.1, n_estimators=100, subsample=1.0, criterion=’friedman_mse’, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, presort=’auto’, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001)

In [45]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from time import time

class_weights = {0:1,1:5}
mod1 = SVC(C=1.0, kernel='rbf', class_weight=class_weights)
mod2 = RandomForestClassifier(class_weight=class_weights)
mod3 = GradientBoostingClassifier()

## 3.1 SVC

In [ ]:
t0=time()
mod1.fit(X_train_vect, y_train)
print("Training time: {} seconds".format((time()-t0)/60))

In [ ]:
mod1_preds = mod1.predict(X_test_vect)
print(classification_report(y_test,mod1_preds))

## 3.2 RandomForestClassifier

In [ ]:
t0=time()
mod2.fit(X_train_vect, y_train)
print("Training time: {} seconds".format((time()-t0)/60))

In [ ]:
mod2_preds = mod2.predict(X_test_vect)
print(classification_report(y_test,y_mod2_preds))

## 3.3 GradientBoostingTrees

In [ ]:
t0=time()
mod3.fit(X_train_vect, y_train)
print("Training time: {} seconds".format((time()-t0)/60))

In [ ]:
mod3_preds = mod3.predict(X_test_vect)
print(classification_report(y_test,mod3_preds))


In [ ]:
#from sklearn.decomposition import TruncatedSVD
#svd = TruncatedSVD(algorithm='randomized', n_components=4400, n_iter=7, random_state=42, tol=0.0)
#data_reduced = svd.fit(data)
#print(svd.explained_variance_ratio_)  # doctest: +ELLIPSIS
#print(svd.explained_variance_ratio_.sum())  # doctest: +ELLIPSIS
#print(svd.singular_values_)  # doctest: +ELLIPSIS


# 4. Test data

In [54]:
df_test = pd.read_csv(f'{PATH}/test.csv')
X_test_final = df_test['question_text']
X_test_final_vect = vectorizer.transform(X_test_final)

In [56]:
X_test_final_vect.shape

(56370, 3009360)